In [50]:
using Pkg
Pkg.activate("/home/molloi-lab/Desktop/Project BAC/BAC project/libs/")
using Lux, Random, NNlib, Zygote, LuxCUDA, CUDA, FluxMPI, JLD2, DICOM
using Images
using Gtk
using Images
using MLUtils
using Optimisers
using ImageMorphology, ChainRulesCore, Statistics, CSV, DataFrames, Dates, imageToolBox
using XLSX, ImageView
full_data_dir  = "/media/molloi-lab/2TB/BAC/Clean_Dataset_full"
output_data_dir = "/media/molloi-lab/2TB2/BAC ver3/subset2"

  Activating project at `~/Desktop/Project BAC/BAC project/libs`


"/media/molloi-lab/2TB2/BAC ver3/subset2"

In [51]:
black_list = ["127715", "101820", "122978", "104591", "133215", "103033","137954", "125865", "105834", "115060"];
for (i, b) in enumerate(black_list)
    black_list[i] = "SID-" * b
end
black_list

10-element Vector{String}:
 "SID-127715"
 "SID-101820"
 "SID-122978"
 "SID-104591"
 "SID-133215"
 "SID-103033"
 "SID-137954"
 "SID-125865"
 "SID-105834"
 "SID-115060"

# Read data

In [52]:
SIDs = readdir(full_data_dir);

In [53]:
xf = XLSX.readxlsx("files/Ca mass calculation Final.xlsx")
SIDs_xf = xf["Final!C:C"][2:end]
L_CC_mass_xf = xf["Final!O:O"][2:end]
L_MLO_mass_xf = xf["Final!S:S"][2:end]
R_CC_mass_xf = xf["Final!F:F"][2:end]
R_MLO_mass_xf = xf["Final!J:J"][2:end]
xf = nothing

sorted_indices = sortperm(SIDs_xf)
SIDs_xf = SIDs_xf[sorted_indices]
L_CC_mass_xf = L_CC_mass_xf[sorted_indices]
L_MLO_mass_xf = L_MLO_mass_xf[sorted_indices]
R_CC_mass_xf = R_CC_mass_xf[sorted_indices]
R_MLO_mass_xf = R_MLO_mass_xf[sorted_indices]

s_SIDs_xf = size(SIDs_xf)[1]

5150

In [54]:
SID_Mass_Dict = Dict()
for i = 1 : size(SIDs_xf)[1]
    SID = SIDs_xf[i]
    mass = (L_CC_mass_xf[i] + L_MLO_mass_xf[i] + R_CC_mass_xf[i] + R_MLO_mass_xf[i])/2f0
    SID_Mass_Dict[SID] = mass
end

In [55]:
buckets_ = [[],[],[],[],[],[]]
buckets = [[],[],[],[],[],[]]
for SID in SIDs
    if !(SID in black_list)
        mass = SID_Mass_Dict[SID]
        idx = min(6, round(Int, ceil(mass/20)+1))
        push!(buckets_[idx], SID)
    end
end

In [56]:
ct = 0 
for i = 1 : 6
    curr = size(buckets_[i])[1]
    ct += curr
    println("$i: $curr")
end
println("Total = $ct")

1: 3788
2: 1108
3: 115
4: 40
5: 31
6: 18
Total = 5100


In [57]:
ct = 0 
for i = 1 : 6
    curr = size(buckets_[i])[1]
    ct += curr
    println("$i: $curr")
end
println("Total = $ct")

1: 3788
2: 1108
3: 115
4: 40
5: 31
6: 18
Total = 5100


# Pick test set

In [58]:
function generate_random_numbers(n, a, b)
    # Check if n is greater than the range size
    if n > (b - a + 1)
        throw(ArgumentError("Number of samples (n) cannot exceed the range size (b - a + 1)"))
    end
    
    # Generate unique random numbers
    random_numbers = Set{Float64}()
    while length(random_numbers) < n
        push!(random_numbers, rand(a:b))
    end
    
    return round.(Int, collect(random_numbers))
end

generate_random_numbers (generic function with 1 method)

In [59]:
target = 8;

In [60]:
num_each_bucket = target
test_buckets = [[],[],[],[],[],[]]
for i = 1 : 6
    test_buckets[i] = buckets_[i][generate_random_numbers(num_each_bucket * (i == 1 ? 5 : 1), 1, size(buckets_[i])[1])]
    buckets[i] = filter(x -> !(x in test_buckets[i]), buckets_[i])
end
ct = 0 
for i = 1 : 6
    curr = size(test_buckets[i])[1]
    ct += curr
    println("$i: $curr")
end
println("Total = $ct")

1: 40
2: 8
3: 8
4: 8
5: 8
6: 8
Total = 80


In [61]:
ct = 0 
for i = 1 : 6
    curr = size(buckets[i])[1]
    ct += curr
    println("$i: $curr")
end
println("Total = $ct")

1: 3748
2: 1100
3: 107
4: 32
5: 23
6: 10
Total = 5020


# Pick Train set

In [62]:
num_each_bucket = target * 4
train_buckets = [[],[],[],[],[],[]]
for i = 1 : 6
    curr_size = size(buckets[i])[1]
    curr_target = num_each_bucket * (i == 1 ? 5 : 1)

    dup_times = floor(curr_target / curr_size)
    for j = 1 : dup_times
        append!(train_buckets[i], buckets[i])
    end

    residue = curr_target % curr_size    
    if residue > 0
        append!(train_buckets[i], buckets[i][generate_random_numbers(residue, 1, curr_size)])
    end
end

ct = 0 
for i = 1 : 6
    curr = size(train_buckets[i])[1]
    ct += curr
    println("$i: $curr")
end
println("Total = $ct")

1: 160
2: 32
3: 32
4: 32
5: 32
6: 32
Total = 320


# Save

In [63]:
@save "JLD2s/train_buckets.jld2" train_buckets
@save "JLD2s/test_buckets.jld2" test_buckets

# Load

In [64]:
# @load "JLD2s/train_buckets.jld2" train_buckets
# @load "JLD2s/test_buckets.jld2" test_buckets

# Copy and paste

In [65]:
ct = 0 
for i = 1 : 6
    curr = size(unique(train_buckets[i]))[1]
    ct += curr
    println("$i: $curr")
end
println("Train Total = $ct\n")

ct = 0 
for i = 1 : 6
    curr = size(unique(test_buckets[i]))[1]
    ct += curr
    println("$i: $curr")
end
println("Test Total = $ct")

1: 160
2: 32
3: 32
4: 32
5: 23
6: 10
Train Total = 289

1: 40
2: 8
3: 8
4: 8
5: 8
6: 8
Test Total = 80


In [66]:
function delete_files_and_folders(root_dir)
    # Get a list of all files and folders under the root directory
    entries = readdir(root_dir)
    
    for entry in entries
        entry_path = joinpath(root_dir, entry)
        if isfile(entry_path)
            # If the entry is a file, delete it
            rm(entry_path)
            # println("Deleted file: $entry_path")
        elseif isdir(entry_path)
            # If the entry is a directory, recursively delete its contents
            delete_files_and_folders(entry_path)
            # After deleting all contents, delete the directory itself
            rm(entry_path)
            # println("Deleted directory: $entry_path")
        end
    end
end

delete_files_and_folders (generic function with 1 method)

In [67]:
function copy_paste(name, curr_bucket)
    output_dir = joinpath(output_data_dir, name)
    isdir(output_dir) || mkdir(output_dir)
    isdir(output_dir) && delete_files_and_folders(output_dir)

    for i = 1 : 6
        for SID in curr_bucket[i]
            to_dir = joinpath(output_dir, SID)
            if !isdir(to_dir)
                mkdir(to_dir)
                from_dir = joinpath(full_data_dir, SID)
                @assert isdir(from_dir)

                for f in readdir(from_dir)
                    from_path = joinpath(from_dir, f)
                    to_path = joinpath(to_dir, f)
                    cp(from_path, to_path)
                end
            end
        end
    end
end

copy_paste (generic function with 1 method)

In [68]:
copy_paste("train", train_buckets);

In [69]:
copy_paste("test", test_buckets);

# Check images

In [70]:
function check_images(name, curr_bucket; target_size = (1920, 1080))
    temp = joinpath(output_data_dir, "check", name)
    isdir(temp) || mkdir(temp)
    delete_files_and_folders(temp)
    for i = 1 : 6
        @Threads.threads for j = 1 : size(curr_bucket[i])[1]
            SID = curr_bucket[i][j]
            curr_dir = joinpath(output_data_dir, name, SID)
            images_ = []
            images__ = []
            for f in readdir(curr_dir)
                f_name, f_ext = splitext(f)
                if f_ext == ".dcm"
                    push!(images_, Float32.(Images.load(joinpath(curr_dir, f_name*".png"))))
                    push!(images__, Float32.(Images.load(joinpath(curr_dir, f_name*".mask.png"))))
                end
            end

            to_show_ = imresize(images_[1], target_size)
            for k = 2 : size(images_)[1]
                to_show_ = hcat(to_show_, ones(target_size[1], 1))
                to_show_ = hcat(to_show_, imresize(images_[k], target_size))
            end

            to_show__ = imresize(images__[1], target_size)
            for k = 2 : size(images__)[1]
                to_show__ = hcat(to_show__, ones(target_size[1], 1))
                to_show__ = hcat(to_show__, imresize(images__[k], target_size))
            end

            Images.save(joinpath(output_data_dir, "check", name, SID*".png"), Gray.(vcat(to_show_, ones(1, size(to_show_)[2]), to_show__)))
        end
    end
end

check_images (generic function with 1 method)

In [71]:
check_images("train", train_buckets)

In [72]:
check_images("test", test_buckets)

# Make cropped images

In [76]:
function delete_files_and_folders(root_dir)
    # Get a list of all files and folders under the root directory
    entries = readdir(root_dir)
    
    for entry in entries
        entry_path = joinpath(root_dir, entry)
        if isfile(entry_path)
            # If the entry is a file, delete it
            rm(entry_path)
            # println("Deleted file: $entry_path")
        elseif isdir(entry_path)
            # If the entry is a directory, recursively delete its contents
            delete_files_and_folders(entry_path)
            # After deleting all contents, delete the directory itself
            rm(entry_path)
            # println("Deleted directory: $entry_path")
        end
    end
end

function crop_to_bounding_box(mask, image, ground_truth_mask)
    # Find rows and columns where the mask has value 1
    indices = findall(x -> x == 1, mask)
    if isempty(indices)
        return "Error in breast mask!" # Return the original mask if no 1s are found
    end

    # Extract row and column indices from the CartesianIndex array
    rows = [index[1] for index in indices]
    cols = [index[2] for index in indices]

    # Determine the initial bounding box
    min_row, max_row = minimum(rows), maximum(rows)
    min_col, max_col = minimum(cols), maximum(cols)

    # Adjust dimensions to be divisible by 32
    width, height = max_col - min_col + 1, max_row - min_row + 1
    if width % 32 != 0
        width_adjustment = 32 - (width % 32)
        max_col += width_adjustment
        # Move the bounding box if it exceeds mask dimensions
        if max_col > size(mask, 2)
            min_col = max(1, min_col - (max_col - size(mask, 2)))
            max_col = size(mask, 2)
        end
    end
    if height % 32 != 0
        height_adjustment = 32 - (height % 32)
        max_row += height_adjustment
        # Move the bounding box if it exceeds mask dimensions
        if max_row > size(mask, 1)
            min_row = max(1, min_row - (max_row - size(mask, 1)))
            max_row = size(mask, 1)
        end
    end
    rslt = image[min_row:max_row, min_col:max_col]
    rslt2 = ground_truth_mask[min_row:max_row, min_col:max_col]
    rslt3 = [min_row, max_row, min_col, max_col]

    x, y = size(rslt)
    res = x % 32
    if res != 0
        top_trim = round(Int, res / 2)
        bot_trim = res - top_trim
        rslt = rslt[top_trim+1 : end - bot_trim, 1: end]
        rslt2 = rslt2[top_trim+1 : end - bot_trim, 1: end]
        rslt3[1] += top_trim
        rslt3[2] -= bot_trim
    end

    res = y % 32
    if res != 0
        left_trim = round(Int, res / 2)
        right_trim = res - left_trim
        rslt = rslt[1: end, left_trim+1 : end - right_trim]
        rslt2 = rslt2[1: end, left_trim+1 : end - right_trim]
        rslt3[3] += left_trim
        rslt3[4] -= right_trim
    end

    return rslt, rslt2, rslt3
end

function resize_dicom_image(image, mask, ground_truth_mask, original_spacing; target_spacing = [0.13, 0.13])
    # Calculate the scaling factors
    scale_x = original_spacing[1] / target_spacing[1]
    scale_y = original_spacing[2] / target_spacing[2]

    new_size_x = round(Int, size(image, 1) * scale_y)
    new_size_y = round(Int, size(image, 2) * scale_x)
    # Resample the image
    resized_img = imresize(image, (new_size_x, new_size_y))
    resized_mask = imresize(mask, (new_size_x, new_size_y))
    resized_ground_truth_mask = imresize(ground_truth_mask, (new_size_x, new_size_y))
    return resized_img, resized_mask, resized_ground_truth_mask, [new_size_x, new_size_y]
end

function prepare_image(name, curr_bucket)
    output_dir = joinpath(output_data_dir, name)

    for i = 1 : 6
        curr_SIDs = unique(curr_bucket[i])
        @Threads.threads for j = 1 : size(curr_SIDs)[1]
            SID = curr_SIDs[j]
            curr_dir = joinpath(output_dir, SID)
            out_dir = joinpath(curr_dir, "cropped")
            isdir(out_dir) || mkdir(out_dir)
            delete_files_and_folders(out_dir)

            for f in readdir(curr_dir)
                f_name, f_ext = splitext(f)
                if f_ext == ".dcm"
                    dcm_path = joinpath(curr_dir, f)
                    breast_mask_path = joinpath(curr_dir, f_name*".mask.png")
                    ground_truth_mask_path = joinpath(curr_dir, f_name*".png")

                    ground_truth_mask = Float32.(Images.load(ground_truth_mask_path))
                    breast_mask = Float32.(Images.load(breast_mask_path))
                    dcm_data = dcm_parse(dcm_path)
                    pixel_size = dcm_data[(0x0018, 0x1164)]
                    img = Float32.(dcm_data[(0x7fe0, 0x0010)])
                    original_size = size(img)
                    # resize image based on pixel length
                    img, breast_mask, ground_truth_mask, new_size = resize_dicom_image(img, breast_mask, ground_truth_mask, pixel_size)
                    # normalize image and correct color
                    img = normalize_img(img; mask = breast_mask, invert = true)
                    # crop to breast only
                    img_cropped, ground_truth_mask_cropped, coords = crop_to_bounding_box(breast_mask, img, ground_truth_mask)
                    # I CAN THROW THE 'img_cropped' thing into the finished BAC model.
                    # save resize info to local
                    @save joinpath(curr_dir, f_name*"_resize_info.jld2") original_size new_size coords
                    # check size
                    x, y = size(img_cropped)
                    # if y % 32 != 0
                    #     x_org, y_org = size(img)
                    #     println(i, "\t", ct+1)
                    #     println("($x_org, $y_org)")
                    #     println("($x, $y)\n")
                    # end
                    @assert x % 32 == 0
                    @assert y % 32 == 0
                    
                    #save
                    @save joinpath(out_dir, f_name*".jld2") img_cropped
                    Images.save(joinpath(out_dir, f_name*".png"), Gray.(round.(ground_truth_mask_cropped)))
                end
            end
        end
    end
    GC.gc(true)
end

prepare_image (generic function with 1 method)

In [77]:
prepare_image("train", train_buckets);

In [79]:
prepare_image("test", test_buckets);